In [ ]:
from collections import Counter
from glob import glob

import pandas as pd
from tqdm import tqdm

In [ ]:
file_paths = glob("./data/raw_data/*.csv")

In [ ]:
def read_csv(file_path):
    encodings = ["utf-8", "cp949"]
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, low_memory=False, encoding=encoding)
            return (df, encoding)
        except UnicodeDecodeError:
            continue
        except pd.errors.ParserError:
            return (None, None)

In [ ]:
file_encoding_dict = dict()
file_column_dict = dict()

for file_path in tqdm(file_paths):
    df, encoding = read_csv(file_path)
    
    if df is not None:
        file_encoding_dict[file_path] = encoding
        file_column_dict[file_path] = list(df.columns)
    
    else:
        file_encoding_dict[file_path] = "ParseError"
        file_column_dict[file_path] = "ParseError"

In [ ]:
Counter(list(file_encoding_dict.values()))

In [ ]:
columns_list = list(file_column_dict.values())

In [ ]:
Counter(sum(columns,[]))

In [ ]:
get_first = lambda x: x.split()[0]

preproc = lambda cols: [get_first(col) for col in cols]
Counter(sum(list(map(preproc,columns)),[]))

In [ ]:
get_first = lambda x: x.split()[0]
do_strip = lambda x: x.strip()

preproc = lambda cols: [do_strip(get_first(col)) for col in cols]
Counter(sum(list(map(preproc,columns)),[]))

In [ ]:
get_first = lambda x: x.split()[0]
do_strip = lambda x: x.strip()
drop_bracket = lambda x: x.split("(")[0]

preproc = lambda cols: [drop_bracket(do_strip(get_first(col))) for col in cols]
Counter(sum(list(map(preproc,columns)),[]))